In [1]:
import zstandard as zstd
import json
import pandas as pd
import io

In [2]:
# read a single zst file
def read_zst_to_json(file_path):
    """
    Read a zst compressed file and return a list of JSON objects.
    
    Args:
        file_path (str): Path to .zst file
        
    Returns:
        list: List of parsed JSON objects
    """
    data = []
    print(f"Processing {file_path}...")
    
    try:
        with open(file_path, 'rb') as fh:
            dctx = zstd.ZstdDecompressor()
            with dctx.stream_reader(fh) as reader:
                text_stream = io.TextIOWrapper(reader, encoding='utf-8')
                for line_count, line in enumerate(text_stream, 1):
                    try:
                        json_obj = json.loads(line.strip())
                        data.append(json_obj)
                        
                        # Print progress every 1000 records
                        if len(data) % 1000 == 0:
                            print(f"Processed {len(data)} records...")
                            
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON at line {line_count}: {e}")
                        continue
                        
        print(f"Total records processed: {len(data)}")
        return data
        
    except Exception as e:
        print(f"Error processing file: {e}")
        import traceback
        traceback.print_exc()
        return []

In [3]:
# First check if file exists
file_path = "../../data/Input/Reddit/submissions/hysterectomy_submissions.zst"
import os
print(f"File exists: {os.path.exists(file_path)}")
if os.path.exists(file_path):
    print(f"File size: {os.path.getsize(file_path)} bytes")

# Read the data
submissions_data = read_zst_to_json(file_path)

# Check the result
print("\nFinal check:")
print(f"Length of data: {len(submissions_data)}")
if len(submissions_data) > 0:
    print("First record:", json.dumps(submissions_data[0], indent=2))

File exists: True
File size: 4054982 bytes
Processing ../../data/Input/Reddit/submissions/hysterectomy_submissions.zst...
Processed 1000 records...
Processed 2000 records...
Processed 3000 records...
Processed 4000 records...
Processed 5000 records...
Processed 6000 records...
Processed 7000 records...
Processed 8000 records...
Processed 9000 records...
Processed 10000 records...
Processed 11000 records...
Total records processed: 11510

Final check:
Length of data: 11510
First record: {
  "link_flair_text": null,
  "permalink": "/r/hysterectomy/comments/26z4vk/changes_to_sexual_functioning/",
  "selftext": "If you've had a hysterectomy (removal of uterus, not ovaries), can you please tell whether or  not you've had changes in your sexual functioning, libido, and ability to have orgasms?",
  "media": null,
  "ups": 1,
  "retrieved_on": 1441331902,
  "created_utc": "1401563576",
  "author_flair_text": null,
  "media_embed": {},
  "subreddit": "hysterectomy",
  "domain": "self.hysterecto

In [4]:
# Look at the first record
if submissions_data:
    print(json.dumps(submissions_data[0], indent=2))

{
  "link_flair_text": null,
  "permalink": "/r/hysterectomy/comments/26z4vk/changes_to_sexual_functioning/",
  "selftext": "If you've had a hysterectomy (removal of uterus, not ovaries), can you please tell whether or  not you've had changes in your sexual functioning, libido, and ability to have orgasms?",
  "media": null,
  "ups": 1,
  "retrieved_on": 1441331902,
  "created_utc": "1401563576",
  "author_flair_text": null,
  "media_embed": {},
  "subreddit": "hysterectomy",
  "domain": "self.hysterectomy",
  "hide_score": false,
  "downs": 0,
  "thumbnail": "self",
  "distinguished": null,
  "title": "Changes to sexual functioning",
  "secure_media_embed": {},
  "from_id": null,
  "secure_media": null,
  "url": "http://www.reddit.com/r/hysterectomy/comments/26z4vk/changes_to_sexual_functioning/",
  "created": 1401567176,
  "subreddit_id": "t5_2x8iq",
  "from_kind": null,
  "stickied": false,
  "score": 1,
  "edited": false,
  "quarantine": false,
  "num_comments": 0,
  "author_flair_

In [5]:
# number of records
print(len(submissions_data))


11510


In [6]:
# Sample a few random titles
import random
print("\nRandom sample of 5 titles:")
for item in random.sample(submissions_data, min(5, len(submissions_data))):
    print(f"- {item['title']}")


Random sample of 5 titles:
- Menopause After Hysterectomy - Good Article &amp; Video
- More Active Before Surgery?
- Surgeon says 2 weeks off should be enough but I think I may need more…
- Yeet Day!
- 6 days post-op from TAH &amp; BSO - Part 2 of 2: how it's going


In [7]:
# What defines a unique record?
# Check if all submissions have unique IDs
ids = [item['id'] for item in submissions_data]
unique_ids = len(set(ids))
print(f"\nTotal submissions: {len(submissions_data)}")
print(f"Unique submission IDs: {unique_ids}")


Total submissions: 11510
Unique submission IDs: 11510


In [8]:
# Convert timestamps to integers and handle any potential type mismatches
timestamps = []
for item in submissions_data:
    try:
        timestamp = int(item['created_utc'])  # Convert to integer
        timestamps.append(timestamp)
    except (ValueError, TypeError) as e:
        print(f"Error with timestamp: {item['created_utc']} - {e}")
        continue

# Now process the timestamps
from datetime import datetime
if timestamps:  # Make sure we have valid timestamps
    earliest = datetime.fromtimestamp(min(timestamps))
    latest = datetime.fromtimestamp(max(timestamps))
    print(f"\nDate range of submissions:")
    print(f"Earliest: {earliest}")
    print(f"Latest: {latest}")


Date range of submissions:
Earliest: 2014-05-31 12:12:56
Latest: 2022-12-31 15:59:13


In [9]:
# turn into a pandas dataframe
submissions_df = pd.DataFrame(submissions_data)

In [10]:
# look at the first few rows
submissions_df.head()

,link_flair_text,permalink,selftext,media,ups,retrieved_on,created_utc,author_flair_text,media_embed,subreddit,...,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,media_metadata,poll_data,gallery_data,is_gallery,call_to_action
0,None,/r/hysterectomy/comments/26z4vk/changes_to_sex...,If you've had a hysterectomy (removal of uteru...,None,1.0,1.441332e+09,1401563576,None,{},hysterectomy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,/r/hysterectomy/comments/3mjfz7/spotting/,"Ladies, I have a dumb question for you. I had...",None,1.0,1.450769e+09,1443323627,None,{},hysterectomy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,/r/hysterectomy/comments/49c2k3/hysterectomy/,,None,1.0,1.463452e+09,1457346839,None,{},hysterectomy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,/r/hysterectomy/comments/4bd4dn/girl_talk_time...,,None,1.0,1.463487e+09,1458579592,None,{},hysterectomy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,/r/hysterectomy/comments/52fcak/4_months_post_...,"I had a hysterectomy about 4 months ago, and I...",None,1.0,1.476412e+09,1473699325,None,{},hysterectomy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
submissions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11510 entries, 0 to 11509
Columns: 126 entries, link_flair_text to call_to_action
dtypes: bool(3), float64(37), int64(3), object(83)
memory usage: 10.8+ MB


In [12]:
submissions_df.isnull().sum()

link_flair_text    11510
permalink              0
selftext               0
media              11478
ups                11478
                   ...  
media_metadata     11414
poll_data          11489
gallery_data       11428
is_gallery         11414
call_to_action     11310
Length: 126, dtype: int64

In [13]:
# only keep necessary columns
# Create a DataFrame with only the columns we need
essential_cols = ['id', 'name', 'created_utc', 'title','selftext','subreddit']
submissions_subset = submissions_df[essential_cols]

# Convert created_utc to datetime for easier analysis
submissions_subset['created_utc'] = pd.to_datetime(submissions_subset['created_utc'], unit='s')

# Display info about the DataFrame
print("DataFrame Info:")
submissions_subset.info()

# Show first few rows
print("\nFirst few rows:")
print(submissions_subset.head())

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11510 entries, 0 to 11509
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           11510 non-null  object        
 1   name         10202 non-null  object        
 2   created_utc  11510 non-null  datetime64[ns]
 3   title        11510 non-null  object        
 4   selftext     11510 non-null  object        
 5   subreddit    11510 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 539.7+ KB

First few rows:
       id       name         created_utc  \
0  26z4vk  t3_26z4vk 2014-05-31 19:12:56   
1  3mjfz7  t3_3mjfz7 2015-09-27 03:13:47   
2  49c2k3  t3_49c2k3 2016-03-07 10:33:59   
3  4bd4dn  t3_4bd4dn 2016-03-21 16:59:52   
4  52fcak  t3_52fcak 2016-09-12 16:55:25   

                                          title  \
0                 Changes to sexual functioning   
1                                     S

/var/folders/zj/d9h6gl49125919pfb15tp_gc0000gn/T/ipykernel_83011/3648906931.py:7: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  submissions_subset['created_utc'] = pd.to_datetime(submissions_subset['created_utc'], unit='s')
/var/folders/zj/d9h6gl49125919pfb15tp_gc0000gn/T/ipykernel_83011/3648906931.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submissions_subset['created_utc'] = pd.to_datetime(submissions_subset['created_utc'], unit='s')


In [18]:
# Filter for submissions containing "Essure" in either title or selftext
essure_pattern = r'\bEssure\b|\bessure\b'
essure_submissions = submissions_subset[
    submissions_subset['title'].str.contains(essure_pattern, regex=True, case=False, na=False) |
    submissions_subset['selftext'].str.contains(essure_pattern, regex=True, case=False, na=False)
]
# Display info about the filtered DataFrame
print("Number of submissions about Essure:", len(essure_submissions))

# Show the titles
print("\nTitles containing 'Essure':")
print(essure_submissions[['created_utc', 'title']].to_string())

Number of submissions about Essure: 14

Titles containing 'Essure':
             created_utc                                                                                    title
217  2019-01-17 14:21:32                                     Women who have had hysterectomies at an "early" age.
504  2019-07-28 13:24:40                                        Sunscreen and fibroids? (Oxybenzone and estrogen)
1208 2020-03-03 20:53:51  Robotic LAVH today and feeling surprisingly good! With SURPRISE traveling Essure coils!
2007 2020-09-26 20:03:21                                                           Essure removal = hysterectomy?
2642 2020-12-29 00:48:19                                                                             Any regrets?
5805 2021-12-09 22:49:03                                                        I’m going to be part of the club!
6098 2022-01-05 23:09:10                                       Why is a hysterectomy considered elective surgery?
6180 2022-01-12 20:2

In [15]:
# comments data frame

file_path = "../../data/Input/Reddit/comments/hysterectomy_comments.zst"
comments_data = read_zst_to_json(file_path)

Processing ../../data/Input/Reddit/comments/hysterectomy_comments.zst...
Processed 1000 records...
Processed 2000 records...
Processed 3000 records...
Processed 4000 records...
Processed 5000 records...
Processed 6000 records...
Processed 7000 records...
Processed 8000 records...
Processed 9000 records...
Processed 10000 records...
Processed 11000 records...
Processed 12000 records...
Processed 13000 records...
Processed 14000 records...
Processed 15000 records...
Processed 16000 records...
Processed 17000 records...
Processed 18000 records...
Processed 19000 records...
Processed 20000 records...
Processed 21000 records...
Processed 22000 records...
Processed 23000 records...
Processed 24000 records...
Processed 25000 records...
Processed 26000 records...
Processed 27000 records...
Processed 28000 records...
Processed 29000 records...
Processed 30000 records...
Processed 31000 records...
Processed 32000 records...
Processed 33000 records...
Processed 34000 records...
Processed 35000 re

In [16]:
# Check the first comment to see its structure
if comments_data:
    print("First comment structure:")
    print(json.dumps(comments_data[0], indent=2))
    
    print("\nTotal number of comments:")
    print(len(comments_data))

First comment structure:
{
  "retrieved_on": 1444604587,
  "gilded": 0,
  "ups": 1,
  "score": 1,
  "distinguished": null,
  "author_flair_css_class": null,
  "author": "lasercat13",
  "subreddit": "hysterectomy",
  "created_utc": "1443325744",
  "edited": false,
  "id": "cvfhc36",
  "body": "I would call your Ob/Gyn and ask to make sure. \r\rI had a hysterectomy back in April this year, and had everything removed except my ovaries. So far I haven't had a hint of spotting. And I've felt better than I have in 14 years. I'm 35, and this was the last step in a long line of treatments. \r\rI hope you get some answers and peace of mind! ",
  "link_id": "t3_3mjfz7",
  "author_flair_text": null,
  "controversiality": 0,
  "subreddit_id": "t5_2x8iq",
  "parent_id": "t3_3mjfz7"
}

Total number of comments:
144908


In [17]:
comments_subset = pd.DataFrame(comments_data)[['id', 'subreddit', 'created_utc', 'body','link_id', 'parent_id']]

In [19]:
# Filter comments containing "Essure"
essure_comments = comments_subset[comments_subset['body'].str.contains(essure_pattern, regex=True, case=True, na=False)]

In [20]:
essure_comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 44 to 134577
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           47 non-null     object
 1   subreddit    47 non-null     object
 2   created_utc  47 non-null     object
 3   body         47 non-null     object
 4   link_id      47 non-null     object
 5   parent_id    47 non-null     object
dtypes: object(6)
memory usage: 2.6+ KB


In [21]:
# merge the comments and submissions
# left join the comments to the submissions
merged_df_1 = pd.merge(essure_submissions, comments_subset, left_on='name', right_on='link_id', how='left')

In [22]:
merged_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_x           90 non-null     object        
 1   name           87 non-null     object        
 2   created_utc_x  90 non-null     datetime64[ns]
 3   title          90 non-null     object        
 4   selftext       90 non-null     object        
 5   subreddit_x    90 non-null     object        
 6   id_y           87 non-null     object        
 7   subreddit_y    87 non-null     object        
 8   created_utc_y  87 non-null     object        
 9   body           87 non-null     object        
 10  link_id        87 non-null     object        
 11  parent_id      87 non-null     object        
dtypes: datetime64[ns](1), object(11)
memory usage: 8.6+ KB


In [23]:
# need to rename the column names
merged_df_1.rename(columns={
    'id_x': 'submission_base_id', 
    'id_y': 'comment_base_id',
    'name': 'submission_id', 
    'link_id': 'comment_link_id',
    'body': 'comment_body',
    'created_utc_x': 'submission_created_utc',
    'created_utc_y': 'comment_created_utc',
    'subreddit_x': 'submission_subreddit',
    'parent_id': 'comment_parent_id',
    'title': 'submission_title',
    'selftext': 'submission_selftext',
    'subreddit_y': 'comment_subreddit'
}, inplace=True)

In [24]:
merged_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   submission_base_id      90 non-null     object        
 1   submission_id           87 non-null     object        
 2   submission_created_utc  90 non-null     datetime64[ns]
 3   submission_title        90 non-null     object        
 4   submission_selftext     90 non-null     object        
 5   submission_subreddit    90 non-null     object        
 6   comment_base_id         87 non-null     object        
 7   comment_subreddit       87 non-null     object        
 8   comment_created_utc     87 non-null     object        
 9   comment_body            87 non-null     object        
 10  comment_link_id         87 non-null     object        
 11  comment_parent_id       87 non-null     object        
dtypes: datetime64[ns](1), object(11)
memory usage: 8.6+ 

In [25]:
merged_df_1.head(10)

,submission_base_id,submission_id,submission_created_utc,submission_title,submission_selftext,submission_subreddit,comment_base_id,comment_subreddit,comment_created_utc,comment_body,comment_link_id,comment_parent_id
0,agy7vh,NaN,2019-01-17 14:21:32,"Women who have had hysterectomies at an ""early...","I'm 28 and will he having my uterus, tubes, an...",hysterectomy,NaN,NaN,NaN,NaN,NaN,NaN
1,civrx4,NaN,2019-07-28 13:24:40,Sunscreen and fibroids? (Oxybenzone and estrogen),So I had the lap-assisted vaginal hyst (uterus...,hysterectomy,NaN,NaN,NaN,NaN,NaN,NaN
2,fd1bbd,NaN,2020-03-03 20:53:51,Robotic LAVH today and feeling surprisingly go...,"My background: 45, 2 vaginal deliveries 20/23 ...",hysterectomy,NaN,NaN,NaN,NaN,NaN,NaN
3,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6px5wu,hysterectomy,1601151359,I had a uterine hysterectomy almost 20 years a...,t3_j0cs4e,t3_j0cs4e
4,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6qa92w,hysterectomy,1601155761,I’m just about 8 weeks post op and I can say t...,t3_j0cs4e,t3_j0cs4e
5,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6qbfok,hysterectomy,1601156177,"I did not have Essure, but I did have a lot of...",t3_j0cs4e,t3_j0cs4e
6,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6qcun9,hysterectomy,1601156660,I doubt there are very many women out there wh...,t3_j0cs4e,t3_j0cs4e
7,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6qi40c,hysterectomy,1601158525,I honestly feel like it will be better to have...,t3_j0cs4e,t1_g6qa92w
8,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6qmtj3,hysterectomy,1601160232,"32, uterusless. No regrets.",t3_j0cs4e,t3_j0cs4e
9,j0cs4e,t3_j0cs4e,2020-09-26 20:03:21,Essure removal = hysterectomy?,Hey everyone. I have Essure which I had gotten...,hysterectomy,g6s1tve,hysterectomy,1601178575,I had the essure put in back in 2010 and never...,t3_j0cs4e,t3_j0cs4e


In [26]:
# left join the submissions to the comments
merged_df_2 = pd.merge(essure_comments, submissions_subset, left_on='link_id', right_on='name', how='left')

In [27]:
merged_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_x           47 non-null     object        
 1   subreddit_x    47 non-null     object        
 2   created_utc_x  47 non-null     object        
 3   body           47 non-null     object        
 4   link_id        47 non-null     object        
 5   parent_id      47 non-null     object        
 6   id_y           42 non-null     object        
 7   name           42 non-null     object        
 8   created_utc_y  42 non-null     datetime64[ns]
 9   title          42 non-null     object        
 10  selftext       42 non-null     object        
 11  subreddit_y    42 non-null     object        
dtypes: datetime64[ns](1), object(11)
memory usage: 4.5+ KB


In [28]:
# rename the column names for the second dataframe
merged_df_2.rename(columns={
    'id_x': 'comment_base_id', 
    'id_y': 'submission_base_id',
    'name': 'submission_id', 
    'link_id': 'comment_link_id',
    'body': 'comment_body',
    'created_utc_y': 'submission_created_utc',
    'created_utc_x': 'comment_created_utc',
    'subreddit_y': 'submission_subreddit',
    'parent_id': 'comment_parent_id',
    'title': 'submission_title',
    'selftext': 'submission_selftext',
    'subreddit_x': 'comment_subreddit'
}, inplace=True)

In [29]:
# append the two dataframes
hysterectomy_full_df = pd.concat([merged_df_1, merged_df_2], ignore_index=True)
hysterectomy_full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   submission_base_id      132 non-null    object        
 1   submission_id           129 non-null    object        
 2   submission_created_utc  132 non-null    datetime64[ns]
 3   submission_title        132 non-null    object        
 4   submission_selftext     132 non-null    object        
 5   submission_subreddit    132 non-null    object        
 6   comment_base_id         134 non-null    object        
 7   comment_subreddit       134 non-null    object        
 8   comment_created_utc     134 non-null    object        
 9   comment_body            134 non-null    object        
 10  comment_link_id         134 non-null    object        
 11  comment_parent_id       134 non-null    object        
dtypes: datetime64[ns](1), object(11)
memory usage: 13.

In [30]:
# drop the duplicate rows
hysterectomy_full_df = hysterectomy_full_df.drop_duplicates()
hysterectomy_full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, 0 to 136
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   submission_base_id      110 non-null    object        
 1   submission_id           107 non-null    object        
 2   submission_created_utc  110 non-null    datetime64[ns]
 3   submission_title        110 non-null    object        
 4   submission_selftext     110 non-null    object        
 5   submission_subreddit    110 non-null    object        
 6   comment_base_id         112 non-null    object        
 7   comment_subreddit       112 non-null    object        
 8   comment_created_utc     112 non-null    object        
 9   comment_body            112 non-null    object        
 10  comment_link_id         112 non-null    object        
 11  comment_parent_id       112 non-null    object        
dtypes: datetime64[ns](1), object(11)
memory usage: 11.7+ KB

In [51]:
# Convert comment_created_utc to datetime
hysterectomy_full_df['comment_created_utc'] = pd.to_datetime(hysterectomy_full_df['comment_created_utc'], unit='s')

# Verify the conversion
print("DataFrame Info after conversion:")
hysterectomy_full_df.info()

# Show a sample to verify both timestamps are in the correct format
print("\nSample of timestamps:")
print(hysterectomy_full_df[['submission_created_utc', 'comment_created_utc']].head())

DataFrame Info after conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 108
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   submission_base_id      94 non-null     object        
 1   submission_id           92 non-null     object        
 2   submission_created_utc  94 non-null     datetime64[ns]
 3   submission_title        94 non-null     object        
 4   submission_selftext     94 non-null     object        
 5   submission_subreddit    94 non-null     object        
 6   comment_base_id         96 non-null     object        
 7   comment_subreddit       96 non-null     object        
 8   comment_created_utc     96 non-null     datetime64[ns]
 9   comment_body            96 non-null     object        
 10  comment_link_id         96 non-null     object        
 11  comment_parent_id       96 non-null     object        
dtypes: datetime64[ns](2), o